# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = "../output_data/cities.csv"
cities_df = pd.read_csv(cities, encoding="ISO-8859-1")
cities_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,souillac,-20.52,57.52,69.80,64,75,17.22,MU,1596434116
1,saint-joseph,-21.37,55.62,69.80,64,90,18.34,RE,1596434116
2,busselton,-33.65,115.33,57.99,64,20,9.69,AU,1596434014
3,luksuhin,14.10,120.88,93.99,58,96,9.17,PH,1596434116
4,batagay,67.63,134.63,69.30,38,99,7.63,RU,1596434116


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = cities_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities_df[["Lat", "Lng"]]

In [5]:
# create map
fig = gmaps.Map()

# add heat layer

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)



fig

Map(configuration={'api_key': None}, data_bounds=[(-40.90002242302239, -179.99999), (85.0, 179.99999)], layers…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities to fit ideal weather conditions (60-70 degrees, wind speed less than 10, cloudiness 0)
ideal_df = cities_df.loc[(cities_df["Max Temp"] > 60) & (cities_df["Max Temp"] < 75) & (cities_df["Wind Speed"] < 10) & (cities_df["Cloudiness"] == 0), :]

#drop any rows that don't contain all three conditions
ideal_df.reset_index(inplace=True)
del ideal_df['index']

ideal_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,luderitz,-26.65,15.16,60.80,31,0,2.24,NaN,1596434123
1,pidhorodne,48.58,35.11,64.40,72,0,6.71,UA,1596434129
2,zolotkovo,55.53,41.11,62.19,66,0,4.00,RU,1596434135
3,tapes,-30.67,-51.40,60.78,87,0,3.38,BR,1596434139
4,ponta do sol,38.73,-28.27,68.23,82,0,4.94,PT,1596434147


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotellist = []

for i in range(len(ideal_df)):
    lat = ideal_df.loc[i]['Lat']
    lng = ideal_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
ideal_df["Hotel Name"] = hotellist
hotel_df = ideal_df.dropna(how='any')
hotel_df.head()

<ipython-input-7-5421c900cadd>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_df["Hotel Name"] = hotellist


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,pidhorodne,48.58,35.11,64.40,72,0,6.71,UA,1596434129,Dnipro
2,zolotkovo,55.53,41.11,62.19,66,0,4.00,RU,1596434135,Zolotkovo
3,tapes,-30.67,-51.40,60.78,87,0,3.38,BR,1596434139,"Tapes, Rio Grande do Sul"
4,ponta do sol,38.73,-28.27,68.23,82,0,4.94,PT,1596434147,Rosais
5,aksehir,38.36,31.42,69.67,54,0,6.55,TR,1596434147,Akşehir


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
#city = ideal_df["City"]
hotel = ideal_df["Hotel Name"]
#country = ideal_df["Country"]

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig


Map(configuration={'api_key': None}, data_bounds=[(-40.90002242302239, -179.99999), (85.0, 179.99999)], layers…